In [1]:
import os
os.chdir('../')
print(os.getcwd())
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from dataset.dataloader import Dataloader
from dataset.dataset import Dataset

/home/mangelini/Develop/UCY/AirBnB_score_prediction


In [2]:
data = Dataloader(
    listing_path = 'dataset/listings',
    comments_path = 'dataset/comments',
)
df = data.getListings()

scores_conlumns = [ col for col in df.columns if "score" in col ]
other_possible_scores_conlumns = ["price"]
scores = df[scores_conlumns + other_possible_scores_conlumns]
scores["price"] = scores.price.apply(lambda p: float(p.replace("$", "").replace(",", "")))

# loads pre_processed data
with open(f'embeddings/listing_name_embeddings.pkl', 'rb') as f:
    listing_name = pd.read_pickle(f)
    
with open(f'embeddings/listing_description_embeddings.pkl', 'rb') as f:
    listing_description = pd.read_pickle(f)

with open(f'embeddings/listing_neighborhood_overview_embeddings.pkl', 'rb') as f:
    listing_neighborhood = pd.read_pickle(f)

with open(f'embeddings/listing_host_about_embeddings.pkl', 'rb') as f:
    listing_host_about = pd.read_pickle(f)

with open(f'embeddings/listing_processed_data.pkl', 'rb') as f:
    listings_numeric = pd.read_pickle(f)

with open(f'embeddings/comments_embeddings.pkl', 'rb') as f:
    comments = pd.read_pickle(f)

with open(f'embeddings/comments_mean_emb.pkl', 'rb') as f:
    comments_mean = pd.read_pickle(f)

/tmp/ipykernel_31860/1407768690.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores["price"] = scores.price.apply(lambda p: float(p.replace("$", "").replace(",", "")))


In [3]:
print("listing_name:", listing_name.shape)
print("listing_description:", listing_description.shape)
print("listing_neighborhood:", listing_neighborhood.shape)
print("listing_host_about:", listing_host_about.shape)
print("listings_numeric:", listings_numeric.shape)
print("comments:", comments.shape)
print("comments_mean:", comments_mean.shape)

listing_name: (6998, 384)
listing_description: (6998, 384)
listing_neighborhood: (6998, 384)
listing_host_about: (6998, 384)
listings_numeric: (6998, 146)
comments: (6998, 384)
comments_mean: (384,)


In [4]:
# Generate train - Test Split this is done Once to be able to compare results
comments_pd = pd.DataFrame(comments, columns= [ f'comments_average_embeddings_{i}' for i in range(comments.shape[1])])
dataset_pd = pd.concat([listings_numeric, listing_name, listing_description, listing_neighborhood, listing_host_about, comments_pd], axis=1)
dataset_pd.shape

(6998, 2066)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(dataset_pd, scores, test_size=0.2, random_state=42)

In [6]:
dataset_train = Dataset(x_train, y_train)
dataset_test = Dataset(x_test, y_test)

In [7]:
with open(f'dataset/train_dataset.pkl', 'wb') as f:
    pickle.dump(dataset_train, f)

with open(f'dataset/test_dataset.pkl', 'wb') as f:
    pickle.dump(dataset_test, f)

# Pure PD
with open(f'dataset/train_dataset_X.pkl', 'wb') as f:
    pickle.dump(x_train, f)

with open(f'dataset/train_dataset_y.pkl', 'wb') as f:
    pickle.dump(y_train, f)

with open(f'dataset/test_dataset_X.pkl', 'wb') as f:
    pickle.dump(x_test, f)

with open(f'dataset/test_dataset_y.pkl', 'wb') as f:
    pickle.dump(y_test, f)